# Equity Metrics Template: Calculating and Mapping Household Vehicle Ownership
### Directions: run each cell in this notebook to generate an interactive map of census tracts shaded by the percent of households that do not own a vehicle.
#### Use [these instructions](https://jupyter.readthedocs.io/en/latest/install.html) to install Jupyter Notebook and [run cells](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Running%20Code.html)
#### Note: The italicized text explains the code in the box


In [1]:
# remove the "#" below and run this cell if you get an error running the cell below that sets up the work environment
# !pip install mapboxgl

In [2]:
# set up work environment
# if you get an error about missing mapboxgl, then go back and follow the instructions in the first cell
%matplotlib inline
import pandas as pd, numpy as np, matplotlib.pyplot as plt
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point
from scipy import ndimage
import matplotlib.pylab as pylab
import json
import requests
import pprint
import numpy as np
from mapboxgl.viz import *
from mapboxgl.utils import *
import ipywidgets as widgets

#this changes how large figures appear; change to make figures larger or smaller
pylab.rcParams['figure.figsize'] = 10, 8

import warnings 
warnings.filterwarnings('ignore')

You'll need a Mapbox token to create maps later in the workflow. While you can use the token included in this notebook, the better option is to create your own token to avoid hitting any request limits. Replace the token (in red text) below by generating your own token [here](https://docs.mapbox.com/help/how-mapbox-works/access-tokens/)

In [3]:
# mapbox token
token = 'pk.eyJ1IjoicmFjaGVsb20iLCJhIjoiY2puYXE2YnV4NTVmcjNxbjE2ZHdjazVlbCJ9.jZKiF7-Y9c1gIGDvO0SmwA'

### Calculate the Percent of Households with No Vehicle

In [15]:
# the variable "fips" links to a table that includes FIPS codes for all states and counties in the United States
fips = pd.read_csv('https://raw.githubusercontent.com/omrachel/equitymetrics/master/tables/FIPS_Code_Dictionary.txt', header=None,
                   names=['State Name','State Code', 'County Code', 'County Name'],dtype=str,
                   usecols=['State Name','State Code', 'County Code', 'County Name'])

In [16]:
# Select the year of analysis
# Note 1: The year refers to the American Community Survey 5-Year dataset (e.g. 2016 is the 2012 - 2016 5-Year dataset)
# Note 2: You can add more years in the first line of code below
pick_year = widgets.Dropdown(options=['Select Year', '2017', '2016', '2015', '2014'],value='Select Year',description='Year:',disabled=False)
display(pick_year)

Dropdown(description='Year:', options=('Select Year', '2017', '2016', '2015', '2014'), value='Select Year')

In [17]:
# Select the state
state_list = list(fips['State Name'].unique())
pick_state = widgets.Dropdown(options=state_list,value='AL',description='State:',disabled=False)
display(pick_state)

Dropdown(description='State:', index=38, options=('SC', 'LA', 'VA', 'ID', 'IA', 'KY', 'MO', 'OK', 'CO', 'IL', …

In [19]:
# Select the county
county_list = list(fips['County Name'].unique())
pick_county = widgets.Dropdown(options=county_list,value='Autauga County',description='County:',disabled=False)
display(pick_county)

Dropdown(description='County:', index=90, options=('Abbeville County', 'Acadia Parish', 'Accomack County', 'Ad…

In [25]:
# visualize poverty rate by census tract
# set up the api
endpoint = 'https://api.census.gov/data/'
# the year is set by the drop down menu above
year = pick_year.value
# can replace with another dataset: https://api.census.gov/data.html
dataset = 'acs/acs5'
# can replace with other variables: https://api.census.gov/data/2016/acs/acs5/variables.html
tables = 'B08201_001E,B08201_002E'
# the state is set by the drop down menu above
state = fips.loc[fips['State Name'] == pick_state.value, 'State Code'].values[0]
# can replace with other county codes: https://www.census.gov/geo/reference/codes/cou.html
county = fips.loc[fips['County Name'] == pick_county.value, 'County Code'].values[0]
# can replace with another key: https://api.census.gov/data/key_signup.html
key = '743d7a3fd3ab38085c25887ef6bed9a331092929'

# this query gets data for the specified year, dataset, and tables for ALL census tracts in the specified state and county
url = requests.Request('GET', endpoint+year+'/'+dataset+'?get=NAME,'+tables+\
                       '&for=tract:*&in=state:'+state+'&in=county:'+county+'&key='+key).prepare().url

print(url)

https://api.census.gov/data/2017/acs/acs5?get=NAME,B08201_001E,B08201_002E&for=tract:*&in=state:06&in=county:037&key=743d7a3fd3ab38085c25887ef6bed9a331092929


In [28]:
response = requests.get(url)
results = response.text
# view first 500 characters of query to make sure data is correct
# you should see a few rows for the county you selected
# if you see the wrong county, then go back and change your selection above then run this cell again
print(results[:500])

[["NAME","B08201_001E","B08201_002E","state","county","tract"],
["Census Tract 5315.04, Los Angeles County, California","1113","159","06","037","531504"],
["Census Tract 5315.03, Los Angeles County, California","772","87","06","037","531503"],
["Census Tract 2060.31, Los Angeles County, California","2140","293","06","037","206031"],
["Census Tract 1863.01, Los Angeles County, California","925","74","06","037","186301"],
["Census Tract 1831.04, Los Angeles County, California","778","24","06","037


In [29]:
# convert data to dataframe so we can analyze
data = json.loads(results)

# create list of columns to use
# the column names (in red text) can be modified to user preferences
columns = ["Description", "TotalPop","Zero_Vehicle","state", "county", "tract"]

# create dataframe from data and drop first row
zero_vehicle =pd.DataFrame.from_records(data,columns=columns).drop(0)

# view new data frame
zero_vehicle.head()

,Description,TotalPop,Zero_Vehicle,state,county,tract
1,"Census Tract 5315.04, Los Angeles County, Cali...",1113,159,06,037,531504
2,"Census Tract 5315.03, Los Angeles County, Cali...",772,87,06,037,531503
3,"Census Tract 2060.31, Los Angeles County, Cali...",2140,293,06,037,206031
4,"Census Tract 1863.01, Los Angeles County, Cali...",925,74,06,037,186301
5,"Census Tract 1831.04, Los Angeles County, Cali...",778,24,06,037,183104


In [30]:
# convert numeric columns from string to integer
zero_vehicle[["TotalPop","Zero_Vehicle","tract"]] = zero_vehicle[["TotalPop","Zero_Vehicle","tract"]].astype(int)

# add new column for percent of households in census tract with zero vehicle ownership
zero_vehicle["Zero_Vehicle_Percent"] = round((zero_vehicle["Zero_Vehicle"]/zero_vehicle["TotalPop"]*100),2)

### Read in Geographic Data
The next cell reads in Los Angeles County census tracts. The file name (in red text) should be replaced with the geojson file (saved on your commputer) of the geography you are analyzing. Most census tract geographies can be downloaded at the county level from open data portals.

In [31]:
# read in los angeles county census tracts
la_tracts = gpd.read_file('data/LA_tracts.geojson')
la_tracts.head()

,shape_area,label,x_center,ct10,y_center,geoid10,shape_len,geometry
0,4025735684.42,9110.01,6620403.0,911001,1998891.0,06037911001,353933.808192,(POLYGON ((-117.6671211134298 34.5580081383781...
1,2078689856.02,9800.03,6575300.0,980003,2112006.0,06037980003,273188.86321,(POLYGON ((-117.8806120004425 34.7636159996687...
2,11118018325.1,9303.01,6603027.0,930301,1932124.0,06037930301,628603.531323,(POLYGON ((-117.6552358388393 34.3972219664572...
3,4824001.88224,5730.03,6500215.0,573003,1747305.0,06037573003,9050.00845755,(POLYGON ((-118.1992330000361 33.7971229999225...
4,6697030.7108,2976.02,6473372.0,297602,1719119.0,06037297602,12308.3153848,(POLYGON ((-118.2879799997457 33.7225829999421...


In [32]:
# remove columns we don't  need
la_tracts = la_tracts.drop(labels=['shape_area','label','x_center','y_center','shape_len'], axis=1)
# rename column name for better readability
la_tracts = la_tracts.rename({'ct10':'tract'}, axis=1)
# view our edited dataframe
la_tracts.head()

,tract,geoid10,geometry
0,911001,06037911001,(POLYGON ((-117.6671211134298 34.5580081383781...
1,980003,06037980003,(POLYGON ((-117.8806120004425 34.7636159996687...
2,930301,06037930301,(POLYGON ((-117.6552358388393 34.3972219664572...
3,573003,06037573003,(POLYGON ((-118.1992330000361 33.7971229999225...
4,297602,06037297602,(POLYGON ((-118.2879799997457 33.7225829999421...


In [33]:
# convert data type to join datasets (next cell)
la_tracts['tract'] = la_tracts['tract'].astype(int)

In [34]:
# join our dataset about the number of households with no vehicle to the geographic dataset
la_zero_vehicle = la_tracts.merge(zero_vehicle, on='tract')
# drop rows with null values to avoid mapping issues
la_zero_vehicle = la_zero_vehicle.dropna(subset=['Zero_Vehicle_Percent'])
# view our newly merged dataset
la_zero_vehicle.head()

,tract,geoid10,geometry,Description,TotalPop,Zero_Vehicle,state,county,Zero_Vehicle_Percent
0,911001,06037911001,(POLYGON ((-117.6671211134298 34.5580081383781...,"Census Tract 9110.01, Los Angeles County, Cali...",1573,67,06,037,4.26
2,930301,06037930301,(POLYGON ((-117.6552358388393 34.3972219664572...,"Census Tract 9303.01, Los Angeles County, Cali...",223,13,06,037,5.83
3,573003,06037573003,(POLYGON ((-118.1992330000361 33.7971229999225...,"Census Tract 5730.03, Los Angeles County, Cali...",594,43,06,037,7.24
4,297602,06037297602,(POLYGON ((-118.2879799997457 33.7225829999421...,"Census Tract 2976.02, Los Angeles County, Cali...",1673,64,06,037,3.83
5,576301,06037576301,(POLYGON ((-118.1850160000802 33.7825970000275...,"Census Tract 5763.01, Los Angeles County, Cali...",1750,298,06,037,17.03


In [35]:
# drop columns we don't need
la_zero_vehicle = la_zero_vehicle.drop(labels=['state','county','Description','TotalPop','Zero_Vehicle'], axis=1)
# rename columns for better readability 
la_zero_vehicle = la_zero_vehicle.rename({'tract':'Cenus Tract','geoid10':'FIPS Code', 'geometry':'geometry', 'Zero_Vehicle_Percent':'Percent of Households with No Vehicle'},axis=1)
# view updated dataset again
la_zero_vehicle.head()

,Cenus Tract,FIPS Code,geometry,Percent of Households with No Vehicle
0,911001,06037911001,(POLYGON ((-117.6671211134298 34.5580081383781...,4.26
2,930301,06037930301,(POLYGON ((-117.6552358388393 34.3972219664572...,5.83
3,573003,06037573003,(POLYGON ((-118.1992330000361 33.7971229999225...,7.24
4,297602,06037297602,(POLYGON ((-118.2879799997457 33.7225829999421...,3.83
5,576301,06037576301,(POLYGON ((-118.1850160000802 33.7825970000275...,17.03


In [36]:
# view descriptive stats for our dataset
# we are only interested in descriptive stats for the "Percent of Households wiht No Vehicle" column
la_zero_vehicle.describe()

,Cenus Tract,Percent of Households with No Vehicle
count,2318.000000,2318.000000
mean,402579.767903,9.427709
std,222563.229941,9.011881
min,101110.000000,0.000000
25%,211141.750000,3.352500
50%,404401.500000,6.775000
75%,551275.500000,12.377500
max,980033.000000,86.840000


In [37]:
# save data to a file
# Note: this file can be uploaded to Carto for mapping
la_zero_vehicle.to_file('data/la_zero_vehicle.geojson', driver='GeoJSON')

### Creating Choropleth Maps
A few notes...
There are two ways to generate choropleth maps. Option 1 is more straightforward but finicky - the map doesn't always show. Option 2 requires a few more steps, but it is more reliable. Detailed instructions for Option 2 are included below.

In [45]:
# OPTION 1
# not always reliable: depending on your commputer, you may get errors that are difficult to provide generalized help here
# create a map with census tracts shaded by the percent of households with no vehicle
viz = ChoroplethViz('data/la_zero_vehicle.geojson',
                    access_token=token,
                    color_property='Percent of Households with No Vehicle ',
                    color_stops=create_color_stops([0, 5, 10, 15,90,100], colors='YlOrRd'),
                    color_function_type='interpolate',
                    line_stroke='--',
                    line_color='grey',
                    line_width=0.25,
                    opacity=0.8,
                    center=(-118.265986,34.036506),
                    zoom=8,
                    below_layer='waterway-label',
                    legend_layout='horizontal',
                    legend_key_shape='bar',
                    legend_key_borders_on=False)
viz.show()

Option 2: Uploading your geojson file to GitHub
1. Upload to your GitHub project folder: You'll see la_zero_vehicle.geojson [here](https://github.com/omrachel/equitymetrics)
2. Click on the file to view in browser: example [here](https://github.com/omrachel/equitymetrics/blob/master/la_zero_vehicle.geojson)
3. Modify the file url to the raw url by deleting 'https://github.com' in the url and replacing it with 'https://raw.githubusercontent.com'
4. Replace the url in the script below with your url

In [46]:
# OPTION 2
# The only text below to change is the url (in red text just after 'ChoroplethViz(')
viz_online = ChoroplethViz('https://raw.githubusercontent.com/omrachel/equitymetrics/master/la_zero_vehicle.geojson', 
                    access_token=token,
                    color_property='Percent of Households with No Vehicle',
                    color_stops=create_color_stops([0, 5, 10, 15,90,100], colors='YlOrRd'),
                    color_function_type='interpolate',
                    line_stroke='-',
                    line_color='grey',
                    line_width=0.25,
                    opacity=0.8,
                    center=(-118.265986,34.036506),
                    zoom=8,
                    below_layer='waterway-label',
                    legend_layout='horizontal',
                    legend_key_shape='bar',
                    legend_key_borders_on=False)
viz_online.show()

In [47]:
# save map as an HTML to view in browser
with open('viz_novehicle.html', 'w') as f:
    f.write(viz_online.create_html())